# Web スクレイピング （Amazon版）
【おことわり】プログラムでアクセスすると回線を切られるので、Selenium により Web ブラウザを遠隔操作することでアクセス、情報の入手を行います。



*   Selenium 入門 --> https://www.selenium.dev/ja/documentation/webdriver/getting_started/



# 1. Selenium による Web スクレイピング

### ライブラリのインストール

In [11]:
import os
os.environ['https_proxy'] = 'http://cmproxy.nda.ac.jp:8080'
!pip install selenium --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org

### ライブラリのインポート

In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

import re
import pandas as pd
from datetime import datetime, timedelta
import urllib.parse

#from bs4 import BeautifulSoup
import time

### Chrome Driver の設定

In [104]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行したい場合は設定
#options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')

### Web ページの取得

In [105]:
url = 'https://www.amazon.co.jp'
query = 'パソコン'
#インスタンス化
driver = webdriver.Chrome(options=options)
#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)
# ページの取得
driver.get(url)

# 要素が表示されるまで待機
WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, '//*[@id="navBackToTop"]/div/span')))
time.sleep(3)

element = driver.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]')
element.send_keys("")
for i in query:
    element.send_keys(i)
    time.sleep(1)
element.send_keys(Keys.ENTER)

win_height = driver.execute_script("return window.innerHeight")

#スクロール開始位置の初期値（ページの先頭からスクロールを開始する）
last_top = 1

#スクロール前のページの高さを取得
last_height = driver.execute_script("return document.body.scrollHeight")

#スクロール開始位置を設定
top = last_top

#ページ最下部まで、徐々にスクロールしていく
while top < last_height:
    top += int(win_height * 0.1)
    driver.execute_script("window.scrollTo(0, %d)" % top)
    time.sleep(0.1)

prices = driver.find_elements(By.CLASS_NAME,"a-price-whole")
print(len(prices), '個の価格を取得')

74 個の価格を取得


### 価格情報の入手

In [103]:
sum = 0
count = 0
for i in prices:
    print(i.text, ',', len(i.text))
    if len(i.text) > 0:
        sum += int(i.text.replace(',', ''))
        count += 1

print("平均価格：", sum / count, '円 (', sum, '円/', count, '個)')

 , 0
 , 0
 , 0
12,800 , 6
31,499 , 6
29,999 , 6
157,000 , 7
12,800 , 6
10,880 , 6
16,800 , 6
164,220 , 7
94,800 , 6
211,000 , 7
 , 0
 , 0
21,000 , 6
17,800 , 6
9,999 , 5
23,800 , 6
33,800 , 6
21,930 , 6
28,800 , 6
25,000 , 6
9,700 , 5
31,800 , 6
79,800 , 6
20,120 , 6
26,800 , 6
25,800 , 6
24,500 , 6
25,800 , 6
10,800 , 6
12,226 , 6
28,866 , 6
21,800 , 6
14,800 , 6
39,800 , 6
23,800 , 6
100,082 , 7
34,999 , 6
13,400 , 6
33,999 , 6
19,800 , 6
49,999 , 6
22,800 , 6
130,901 , 7
12,800 , 6
41,800 , 6
19,800 , 6
42,800 , 6
35,800 , 6
31,840 , 6
35,917 , 6
31,280 , 6
31,000 , 6
23,800 , 6
24,800 , 6
33,800 , 6
123,310 , 7
39,700 , 6
21,800 , 6
63,980 , 6
25,330 , 6
19,500 , 6
19,990 , 6
27,500 , 6
44,800 , 6
24,508 , 6
31,840 , 6
25,330 , 6
54,998 , 6
95,000 , 6
 , 0
 , 0
平均価格： 40437.94029850746 円 ( 2709342 円/ 67 個)


### ブラウザを閉じる

In [ ]:
driver.quit()